<a href="https://colab.research.google.com/github/roesta07/How-Business-Strategies-can-create-bias/blob/main/strategy_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Imports and utility functions
# Install packages that are not installed in colab
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    !pip install pymc3==3.9
    !pip install --upgrade daft
    !pip install --upgrade theano
    !pip install causalgraphicalmodels
    !pip install watermark
    !pip install arviz --no-dependencies
    !pip install netCDF4
    !pip install xarray

     |████████████████████████████████| 1.9MB 7.6MB/s 
     |████████████████████████████████| 1.6MB 40.2MB/s 
     |████████████████████████████████| 4.7MB 45.2MB/s 
     |████████████████████████████████| 737kB 40.0MB/s 
     |████████████████████████████████| 102kB 9.4MB/s 
     |████████████████████████████████| 317kB 42.0MB/s 
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7667 sha256=0be2d00bb6c5698cee77a32ff30999c2110831d33724fda3cd5660fff0a246d9
  Stored in directory: /root/.cache/pip/wheels/a5/7d/68/1ebae2668bda2228686e3c1cf16f2c2384cea6e9334ad5f6de
Successfully built contextvars
  Found existing installation: xarray 0.15.1
    Uninstalling xarray-0.15.1:
      Successfully uninstalled xarray-0.15.1
  Found existing installation: pymc3 3.7
    Uninstalling pymc3-3.7:
      Successfully uninstalled pymc3-3.7
  Created wheel for daft: filename=daft-0.1.0-cp36-none-any.whl size=11375 sha256=0e9f7a30a79ec3388bb4d3a132e869a4c4a98f4716eeac837496f738

In [3]:
import arviz as az
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import pymc3 as pm
import scipy.stats as stats
import daft
from theano import theano as shared

from scipy.interpolate import griddata
from causalgraphicalmodels import CausalGraphicalModel

In [4]:
%config InlineBackend.figure_format = 'retina'
az.style.use('arviz-darkgrid')
# %load_ext watermark
az.rcParams["stats.hdi_prob"] = 0.89  # sets default credible interval used by arviz€

In [41]:
## business simulation
b_CO=1
b_UO=1
b_OP=2
b_UP=1
b_CP=0



n_samples=500
C=np.random.normal(10,2,size=n_samples)
## which are regeions
## east or west
U = 2*np.random.binomial(1,0.7,size=n_samples)-1
O = np.random.normal(b_CO*C+b_UO*U,1,size=n_samples)
P=  np.random.normal(b_UP*U+b_OP*O+b_CP*C,2,size=n_samples)


In [42]:
## lets create a dataframe
df=pd.DataFrame(np.vstack([C,O,P,U]).T,columns=['C','O','P','U'])
df.head()

,C,O,P,U
0,8.460415,9.269408,15.734126,-1.0
1,12.966366,13.931534,27.791250,-1.0
2,11.698770,9.458520,16.561038,-1.0
3,10.329245,11.531452,25.023763,1.0
4,8.163500,8.943648,23.725636,1.0


In [37]:
## Some number transformations

def std_standarize(col):
  '''Takes in a numpy array, returns its standard unit'''
  return (col-col.mean())/col.std()

def normalize(col):
  '''Takes in a numpy array, returns min-max scale'''
  return col/col.max()

df=df.assign(C_std=std_standarize(df['C']),O_norm=normalize(df['O']),P_std=std_standarize(df['P']))
print(df['P_std'].mean())
df.head()


1.538547067525542e-15


,C,O,P,U,C_std,O_norm,P_std
0,11.991669,11.151960,20.206274,1.0,0.965669,0.642646,-0.211756
1,8.075194,8.329097,17.506015,1.0,-0.972063,0.479975,-0.667067
2,8.698336,10.508111,24.906674,1.0,-0.663755,0.605543,0.580814
3,11.450184,11.886631,26.651850,1.0,0.697762,0.684982,0.875081
4,9.129323,9.524547,20.464955,1.0,-0.450518,0.548864,-0.168138


In [38]:
## Modeling
with pm.Model() as m_1:
  a=pm.Normal('a',0,0.2)
  bCP=pm.Normal('b_CP',0,0.5)
  b_OP=pm.Normal('b_OP',0,1)
  sigma=pm.Exponential('sigma',1)
  mu=pm.Deterministic('mu',a+b_CP*(df['C_std'])+b_OP*(df['O_norm']))
  profit=pm.Normal('profit',mu=mu,sigma=sigma,observed=df['P_std'])
  trace_1=pm.sample()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [sigma, b_OP, b_CP, a]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 10 seconds.


In [39]:
az.summary(trace_1,var_names=['~mu'])

/usr/local/lib/python3.6/dist-packages/arviz/data/io_pymc3.py:92: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


,mean,sd,hdi_5.5%,hdi_94.5%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
a,-2.691,0.106,-2.857,-2.523,0.003,0.002,977.0,977.0,985.0,1061.0,1.0
b_CP,0.005,0.501,-0.835,0.747,0.013,0.011,1410.0,959.0,1410.0,1235.0,1.0
b_OP,4.534,0.172,4.270,4.814,0.006,0.004,947.0,946.0,952.0,1184.0,1.0
sigma,0.581,0.020,0.552,0.615,0.001,0.000,1206.0,1206.0,1197.0,1191.0,1.0
